## Homework

У цьому завданні ми розгорнемо модель **Bank Marketing** з домашнього завдання №5.  
У нас вже є Docker-образ для цієї моделі – ми використаємо його для  
розгортання моделі у **Kubernetes**.



### Створення образу

Клонуйте репозиторій курсу, якщо ви ще цього не зробили:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Перейдіть у папку `course-zoomcamp/cohorts/2024/05-deployment/homework`  
і виконайте наступну команду:

```bash
docker build -t zoomcamp-model:3.11.5-hw10 .
```

> **Примітка:** Якщо виникли проблеми зі збіркою образу, ви можете  
> скористатися образом, який ми зібрали і опублікували на **Docker Hub**:  
> `docker pull svizor/zoomcamp-model:3.11.5-hw10`

In [1]:
!docker pull svizor/zoomcamp-model:3.11.5-hw10

3.11.5-hw10: Pulling from svizor42/zoomcamp-model

a5fb9032: Pulling fs layer 
81b69b9a: Pulling fs layer 
a0a62ab5: Pulling fs layer 
28511179: Pulling fs layer 
79bc1c47: Pulling fs layer 
c3f7336a: Pulling fs layer 
56631b9d: Pulling fs layer 
f65bff9f: Pulling fs layer 
56631b9d: Waiting fs layer 
8ef822c9: Pulling fs layer 
Digest: sha256:a33db66766aa612128df9d4e24ad9d4b1b582bb51ae2e6b23f78b0edad9ffbd2
Status: Downloaded newer image for svizor/zoomcamp-model:3.11.5-hw10
docker.io/svizor/zoomcamp-model:3.11.5-hw10


In [ ]:
!docker images svizor/zoomcamp-model:3.11.5-hw10

## Question 1

Запустіть його для локального тестування:

In [ ]:
!docker run -it --rm -p 9696:9696 svizor/zoomcamp-model:3.11.5-hw10



В іншому терміналі виконайте файл `q6_test.py`:

```bash
python q6_test.py
```

Ви побачите таке:

```python
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Тут `<value>` – це ймовірність підписки. Оберіть правильне значення:

* 0.287
* 0.530
* **0.757**
* 0.960


In [8]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

(used install using apt for kubectl, and install from repo for kind)

## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.


In [10]:
!kind --version

kind version 0.17.0


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

In [1]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:39879
CoreDNS is running at https://127.0.0.1:39879/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod
* Deployment
* Service

**Answer: Pod**

# Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* ClusterIP
* NodePort
* LoadBalancer
* ExternalName


In [7]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   4m55s


**Answer is: The service already running is ClusterIP**

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:v001`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`


In [10]:
!kind load docker-image svizor42/zoomcamp-model:3.11.5-hw10


Image: "" with ID "sha256:951a2ef903b849d0ed069942d05aa2edd5e50cb7f91ceb9a7301068d0910a18c" not yet present on node "kind-control-plane", loading...


**Answer: kind load docker-image**

## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "250m"
        ports:
        - containerPort: 9696
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?


Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

```
        image: zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "250m"
        ports:
        - containerPort: 9696
```

**Answer: The value for port is 80**

In [11]:
# !kubectl get pods --all-namespaces
!kubectl apply -f deployment.yaml

deployment.apps/subscription created


In [12]:
!kubectl get deployment

NAME          READY   UP-TO-DATE   AVAILABLE   AGE
subscription   1/1     1            1           22s


Yes there is one running pod as expected

In [13]:
!kubectl get pod

NAME                           READY   STATUS    RESTARTS   AGE
subscription-56b8f7f4d8-qffv8   1/1     Running   0          2m35s


## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

**Answer: subscription**

Apply this config file.

my service.yaml:

```
apiVersion: v1
kind: Service
metadata:
  name: subscription-service
spec:
  type: NodePort
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696
```

In [16]:
!kubectl apply -f service.yaml

service/subscription-service created


In [17]:
!kubectl get service

NAME          TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
subscription  LoadBalancer   10.96.15.137   <pending>     80:31883/TCP   25s
kubernetes    ClusterIP      10.96.0.1      <none>        443/TCP        144m


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/subscribtion-service 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [18]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


# Autoscaling

Now we're going to use a[HorizontalPodAutoscaler](https: // kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/)
(HPA for short) that automatically updates a workload resource(such as our deployment),
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME               REFERENCE                TARGETS          MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   <unknown>/20%   1         3         1          26s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0 % as there are no clients sending requests to the server.
>
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release
> from the `components.yaml` manifest:
> ```bash
> kubectl apply - f https: // github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```


In [19]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS         MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   <unknown>/20%   1         3         0          5s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the subscription service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

In [ ]:
!python q6_test.py

**Observation: It returns the output in a loop.**

## Question 8 (optional)

Run `kubectl get hpa subscription -hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* **1**
* 2
* 3
* 4

```bash
NAME               REFERENCE                TARGETS          MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   <unknown>/20%   1         3         1          7m19s
```

>Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.
